In [2]:
import os
import json
import time
import random
import numpy as np
from datetime import datetime, timedelta
from osgeo import gdal
from matplotlib import pyplot as plt

In [3]:
DATA_ROOT_DIR = "/Volumes/X/Data/fusion-s1-s2/"

In [4]:
CROP_SIZE = 256

In [23]:
ORBIT = "044"
S1_ROOT_PATH = f"{DATA_ROOT_DIR}s1db/32VNH/threeband/{ORBIT}/"

In [16]:
im = gdal.Open(S1_ROOT_PATH + "S1_32VNH_20190103_A_044.tif")
data = np.transpose(im.ReadAsArray(), (1, 2, 0))[:,:,:2]

In [48]:
band_map = {
    "HV": 1,
    "VV": 2
}

In [51]:
for s1 in os.listdir(S1_ROOT_PATH):
    image_path = f"{S1_ROOT_PATH}{s1}"
    name = "_".join(s1.split("_")[:3])
    image = gdal.Open(image_path)
    print(f"Processing {image_path}")

    width = image.RasterXSize
    height = image.RasterYSize

    gt = image.GetGeoTransform()

    min_x = int(gt[0])
    min_y = int(gt[3]) - CROP_SIZE*10
    max_x = int(gt[0] + width*gt[1])
    max_y = int(gt[3] + height*gt[5]) - CROP_SIZE*10

    x_length = range(min_x, max_x + CROP_SIZE + 1, CROP_SIZE*10)
    y_length = range(min_y, max_y - CROP_SIZE + 1, -CROP_SIZE*10)

    ALL_XY_COORDINATES = [(x, y) for y in y_length for x in x_length]

    for band in ["HV", "VV"]:
        output_vrt_path = f"data/cropped/s1/{name}_{band}.vrt"
        gdal.Translate(output_vrt_path, image_path, format="VRT", bandList=[band_map[band]])
        
        # for idx, (x, y) in enumerate(ALL_XY_COORDINATES):
        #     if not os.path.exists(f"data/cropped/s1/{name}/{name}_{band}"):
        #         os.makedirs(f"data/cropped/s1/{name}/{name}_{band}")
            
        #     vrt_options = gdal.BuildVRTOptions(resolution="highest", outputBounds=(x, y, x + CROP_SIZE * 10, y + CROP_SIZE * 10))
    break

Processing /Volumes/X/Data/fusion-s1-s2/s1db/32VNH/threeband/044/S1_32VNH_20190103_A_044.tif
